In [1]:
!lscpu

Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          48 bits physical, 48 bits virtual
  Byte Order:             Little Endian
CPU(s):                   8
  On-line CPU(s) list:    0-7
Vendor ID:                AuthenticAMD
  Model name:             AMD EPYC 7B12
    CPU family:           23
    Model:                49
    Thread(s) per core:   2
    Core(s) per socket:   4
    Socket(s):            1
    Stepping:             0
    BogoMIPS:             4499.99
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush mmx fxsr sse sse2 ht syscall
                           nx mmxext fxsr_opt pdpe1gb rdtscp lm constant_tsc rep
                          _good nopl nonstop_tsc cpuid extd_apicid tsc_known_fre
                          q pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic mo
                          vbe popcnt aes xsave avx f16c rdrand hypervisor lahf_l

**----------Load The Data---------**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import time
from collections import deque
from tqdm import tqdm
from typing import Dict, List, Tuple
import math
import random
import numpy as np
from numpy import ndarray

In [4]:
# Set seeds for reproducibility
# --- Fixed parameters ---
dim = 7 # dimension of the points
np.random.seed(42)  # set seed for reproducibility

In [5]:
# Define the dataset path
dataset_path = '/content/drive/MyDrive/datasets/dataSearch/higgs_ds_mod_orig.txt'

# Load the dataset
# If the file has a different delimiter (e.g., commas), you can set delimiter=',' accordingly.
dataset = np.loadtxt(dataset_path)
total_points = len(dataset)

In [6]:
# Data sizes:
SB = 10_000    # Small Batch: 10K points
MB = 100_000   # Medium Batch: 100K points
LB = 1_000_000 # Large Batch: 1M points

# Query sizes:
SQ = 20        # Small Query: 20 queries
MQ = 100       # Medium Query: 100 queries
LQ = 500       # Large Query: 500 queries

# === Define the test combinations ===
# Each tuple is of the form (data_size, query_size)
test_combinations = [
    (LB, SQ)
]

In [25]:
# ----------------------------
# Phase 1: Pre-generate and Store Queries from the Dataset
# ----------------------------

queries_list = []
st_idx = 0  # Starting index for slicing the dataset

for data_size, query_size in test_combinations:
    # Slice the current data chunk
    end_idx = st_idx + data_size
    current_data = dataset[st_idx:end_idx]
    st_idx = end_idx  # Update start index for next chunk

    # Ensure query selection is different for each batch size
    if query_size <= len(current_data):  # Sampling from the current batch only
        random_indices = np.random.choice(len(current_data), size=query_size, replace=False)
        queries = current_data[random_indices]  # Select query points from the current batch
    else:
        raise ValueError(f"Query size {query_size} exceeds available data points {len(current_data)}.")

    # Append the selected queries to the list
    queries_list.append(queries)


**--------QuickSelect Algorithm----------------**

In [8]:
from collections import deque
from typing import List, Tuple

#*-----------------------------------------------------------------------------
def quickselect(distances:List, k:int):
    """QuickSelect algorithm to find the k-th smallest distance."""

    if len(distances) == 1:
        return distances[0]

    if len(distances) == 0:
        return None

    stack = deque()
    stack.append( (distances, k) )
    while stack:

        distances, k = stack.pop()
        if len(distances) == 1:
            return distances[0]

        # Returned as a tuple
        pivot = median_of_medians(distances)
        lows, highs, pivots = partition(distances, pivot)

        if len(lows) > k:
            stack.append((lows, k))
        elif len(lows) + len(pivots) > k:
            return pivot
        else:
            stack.append((highs, k - len(lows) - len(pivots)))
    return pivot

#*-----------------------------------------------------------------------------
def partition(distances, pivot):
    lows, highs, pivots = [], [], []

    for d in distances:
        if d[0] < pivot[0]:
            lows.append(d)
        elif d[0] > pivot[0]:
            highs.append(d)
        else:
            pivots.append(d)
    return lows, highs, pivots

#*-----------------------------------------------------------------------------
def median_of_medians(distances, grp_size=5):
    """Find the index of the median distance in a list of distances.
    Returns the index and the median distance as a tuple."""

    stack = deque()
    # Split the list in chunks of grp_size
    # And add the lists to the stack on the right side
    for i in range(0, len(distances), grp_size):
        subList = distances[i:i+grp_size]
        insertion_sort(subList)
        stack.append(subList)

    # We want to find the median of the medians
    # and return the index of that median
    median_list = []
    stack_count = len(stack)
    while 0 < stack_count:

        subList = stack.pop()
        stack_count -= 1

        mid_index = len(subList) // 2
        median_list.append(subList[mid_index])

        if len(median_list) == grp_size:
            insertion_sort(median_list)
            # append to the left, since we are popping from the right
            stack.appendleft(median_list)
            if stack_count == 0:
                stack_count = len(stack)
            median_list = []
        # If the count is 0, we have reached the end of the old stack
        # and need to insert the remaining elements,
        # stack count and the median list will be reset
        if stack_count == 0 and len(median_list) > 0:
            if len(median_list) > 1:
                insertion_sort(median_list)
            stack.appendleft(median_list)
            stack_count = len(stack)
            if stack_count == 1 and len(median_list) == 1:
                return median_list[0]
            median_list = []

#*-----------------------------------------------------------------------------
def insertion_sort(arr):
# In-place insertion sort, efficient for small arrays
    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1
        while j >= 0 and arr[j][0] > key[0]:
            arr[j + 1] = arr[j]
            j -= 1
        arr[j + 1] = key
#*-----------------------------------------------------------------------------

**----------VP Tree---------------------**

In [9]:
import numpy as np
from collections import deque

class VPNode:
    def __init__(self, point=None):
        self.point = point
        self.mu = 0
        self.threshold_points = []
        self.left_tr = None
        self.right_tr = None
        self.points = []
        self.centroid = None
        self.radius = 0

        # Cache for numpy arrays
        self._np_points = None
        self._np_threshold_points = None

class VPTree:
    def __init__(self, leaf_size=50, epsilon=0.1):
        self.root = None
        self.leaf_size = leaf_size
        self.epsilon = epsilon
        self._prepared = False  # Track if the tree is prepared for search

    def build(self, points):
        self.root = VPNode()
        stack = deque([(self.root, points)])

        while stack:
            node, pts = stack.pop()
            n_pts = len(pts)

            if n_pts <= self.leaf_size:
                node.points = [p for p in pts]
                node.centroid = np.mean(pts, axis=0)
                node.radius = np.max(np.linalg.norm(pts - node.centroid, axis=1))
                continue

            node.point = pts[0]
            remaining_pts = pts[1:]

            if len(remaining_pts) == 0:
                node.points = [node.point]
                node.centroid = node.point
                node.radius = 0
                continue

            distances = np.linalg.norm(remaining_pts - node.point, axis=1)
            median_idx = len(distances) // 2
            dist_tuples = [(float(distances[i]), i) for i in range(len(distances))]
            median_tuple = quickselect(dist_tuples, median_idx)
            node.mu = median_tuple[0]

            threshold_mask = (distances > node.mu - self.epsilon) & (distances < node.mu + self.epsilon)
            threshold_pts = remaining_pts[threshold_mask]

            if len(threshold_pts) > 0 and len(threshold_pts) < self.leaf_size:
                node.threshold_points = [p for p in threshold_pts]
                node.centroid = np.mean(threshold_pts, axis=0)
                node.radius = np.max(np.linalg.norm(threshold_pts - node.centroid, axis=1))

            inner_mask = distances <= node.mu - self.epsilon
            outer_mask = distances >= node.mu + self.epsilon
            inner_pts = remaining_pts[inner_mask]
            outer_pts = remaining_pts[outer_mask]

            if len(inner_pts) > 0:
                node.left_tr = VPNode()
                stack.appendleft((node.left_tr, inner_pts))

            if len(outer_pts) > 0:
                node.right_tr = VPNode()
                stack.appendleft((node.right_tr, outer_pts))

        return self.root

    def prepare_for_search(self):
        """
        Convert all list-based collections to numpy arrays for efficient search.
        This should be called once after all insertions and before searches.
        """
        if self._prepared:
            return  # Only convert once unless new insertions occurred

        stack = [self.root]

        while stack:
            node = stack.pop()

            if node is None:
                continue

            # Convert points to numpy array
            if node.points:
                node._np_points = np.array(node.points)

            # Convert threshold points to numpy array
            if node.threshold_points:
                node._np_threshold_points = np.array(node.threshold_points)

            # Add child nodes to stack
            if node.left_tr:
                stack.append(node.left_tr)
            if node.right_tr:
                stack.append(node.right_tr)

        self._prepared = True

    def search(self, query):
        """
        Performs an efficient nearest neighbor search using pre-converted numpy arrays.
        """
        # Ensure tree is prepared for search
        if not self._prepared:
            self.prepare_for_search()

        best_dist = float("inf")
        best_point = None
        stack = [self.root]

        while stack:
            node = stack.pop()
            if node is None:
                continue

            # Check leaf node points
            if node.points:
                if node.centroid is not None:
                    dist_to_centroid = np.linalg.norm(query - node.centroid)
                    if dist_to_centroid - node.radius >= best_dist:
                        continue

                # Use cached numpy array for fast vector operations
                dists = np.linalg.norm(node._np_points - query, axis=1)
                min_idx = np.argmin(dists)
                if dists[min_idx] < best_dist:
                    best_dist = dists[min_idx]
                    best_point = node._np_points[min_idx]
                continue

            # Check vantage point
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                if dist_to_vp < best_dist:
                    best_dist = dist_to_vp
                    best_point = node.point

            # Check threshold points
            if node.threshold_points:
                dist_to_centroid = np.linalg.norm(query - node.centroid)
                if dist_to_centroid - node.radius < best_dist:
                    # Use cached numpy array for threshold points
                    dists = np.linalg.norm(node._np_threshold_points - query, axis=1)
                    min_idx = np.argmin(dists)
                    if dists[min_idx] < best_dist:
                        best_dist = dists[min_idx]
                        best_point = node._np_threshold_points[min_idx]

            # Determine which subtree to explore
            if node.point is not None:
                dist_to_vp = np.linalg.norm(node.point - query)
                dist_to_threshold = abs(dist_to_vp - node.mu)

                if dist_to_vp <= node.mu - self.epsilon:
                    # Query is in inner region
                    if node.left_tr:
                        stack.append(node.left_tr)
                    if node.right_tr and dist_to_threshold < best_dist:
                        stack.append(node.right_tr)
                elif dist_to_vp >= node.mu + self.epsilon:
                    # Query is in outer region
                    if node.right_tr:
                        stack.append(node.right_tr)
                    if node.left_tr and dist_to_threshold < best_dist:
                        stack.append(node.left_tr)
                else:
                    # Query is in threshold band
                    if dist_to_vp <= node.mu:
                        # Prioritize inner subtree
                        if node.left_tr:
                            stack.append(node.left_tr)
                        if node.right_tr:
                            stack.append(node.right_tr)
                    else:
                        # Prioritize outer subtree
                        if node.right_tr:
                            stack.append(node.right_tr)
                        if node.left_tr:
                            stack.append(node.left_tr)

        return best_dist, best_point

    def insert(self, point):
        """
        Insert a new point into the VP-Tree with fast list-based operations.
        """
        # Mark tree as needing preparation before next search
        self._prepared = False

        if self.root is None:
            self.root = VPNode()
            self.root.points = [point]
            self.root.centroid = point
            self.root.radius = 0
            return

        current = self.root

        while not current.points:
            if current.point is None:
                if current.left_tr:
                    current = current.left_tr
                elif current.right_tr:
                    current = current.right_tr
                else:
                    current.points = [point]
                    current.centroid = point
                    current.radius = 0
                    return
            else:
                dist_to_vp = np.linalg.norm(point - current.point)

                if abs(dist_to_vp - current.mu) <= self.epsilon:
                    if not current.threshold_points:
                        current.threshold_points = [point]
                        current.centroid = point
                        current.radius = 0
                        return
                    elif len(current.threshold_points) < self.leaf_size:
                        n = len(current.threshold_points)
                        current.threshold_points.append(point)
                        current.centroid = (n * current.centroid + point) / (n + 1)
                        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))
                        return

                if dist_to_vp < current.mu - self.epsilon:
                    if current.left_tr:
                        current = current.left_tr
                    else:
                        current.left_tr = VPNode()
                        current.left_tr.points = [point]
                        current.left_tr.centroid = point
                        current.left_tr.radius = 0
                        return
                else:
                    if current.right_tr:
                        current = current.right_tr
                    else:
                        current.right_tr = VPNode()
                        current.right_tr.points = [point]
                        current.right_tr.centroid = point
                        current.right_tr.radius = 0
                        return

        n = len(current.points)
        current.points.append(point)
        current.centroid = (n * current.centroid + point) / (n + 1)
        current.radius = max(current.radius, np.linalg.norm(point - current.centroid))

**------VP Tree Node Size Optimization For Rebuild------**

**------------Run 1---------------------**

In [10]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 41.33it/s]


-------------------------------------------------------------
Build Time: 19.4461 sec, Search Time: 0.7677 sec
Leaf Size: 100, Total Time: 20.2138 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 53.88it/s]


-------------------------------------------------------------
Build Time: 18.0228 sec, Search Time: 0.5723 sec
Leaf Size: 200, Total Time: 18.5951 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 12.7435 sec, Search Time: 0.4784 sec
Leaf Size: 9,800, Total Time: 13.2219 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 64.63it/s]


-------------------------------------------------------------
Build Time: 12.6062 sec, Search Time: 0.4755 sec
Leaf Size: 9,900, Total Time: 13.0817 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 60.81it/s]

-------------------------------------------------------------
Build Time: 12.4449 sec, Search Time: 0.4978 sec
Leaf Size: 10,000, Total Time: 12.9427 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 19.4461 sec, Search Time: 0.7677 sec, Total Time: 20.2138 sec
Leaf Size: 200, Build Time: 18.0228 sec, Search Time: 0.5723 sec, Total Time: 18.5951 sec
Leaf Size: 300, Build Time: 17.7721 sec, Search Time: 0.4650 sec, Total Time: 18.2371 sec
Leaf Size: 400, Build Time: 16.9794 sec, Search Time: 0.4651 sec, Total Time: 17.4445 sec
Leaf Size: 500, Build Time: 16.1929 sec, Search Time: 0.4324 sec, Total Time: 16.6252 sec
Leaf Size: 600, Build Time: 17.1709 sec, Search Time: 0.4204 sec, Total Time: 17.5913 sec
Leaf Size: 700, Build Time: 16.4135 sec, Search Time: 0.4003 sec, Total Time: 16.8138 sec
Leaf Size: 800, Build Time: 16.5635 sec, Search Time: 0.4043 sec, Total Time: 16.9678 sec
Leaf Size: 900, Build Time: 16.5119 sec, Search Time: 0.4016 sec, Total Time: 16.91

**---------------------Run 2---------------**

In [12]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

# Fix the epsilon at 0.0
epsilon=0.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 36.95it/s]


-------------------------------------------------------------
Build Time: 21.3394 sec, Search Time: 0.7893 sec
Leaf Size: 100, Total Time: 22.1287 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 53.00it/s]


-------------------------------------------------------------
Build Time: 18.9936 sec, Search Time: 0.5674 sec
Leaf Size: 200, Total Time: 19.5609 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 12.2762 sec, Search Time: 0.4949 sec
Leaf Size: 9,800, Total Time: 12.7711 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 56.57it/s]


-------------------------------------------------------------
Build Time: 12.5590 sec, Search Time: 0.5186 sec
Leaf Size: 9,900, Total Time: 13.0776 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 59.45it/s]

-------------------------------------------------------------
Build Time: 12.6931 sec, Search Time: 0.4973 sec
Leaf Size: 10,000, Total Time: 13.1904 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 21.3394 sec, Search Time: 0.7893 sec, Total Time: 22.1287 sec
Leaf Size: 200, Build Time: 18.9936 sec, Search Time: 0.5674 sec, Total Time: 19.5609 sec
Leaf Size: 300, Build Time: 18.2972 sec, Search Time: 0.4623 sec, Total Time: 18.7596 sec
Leaf Size: 400, Build Time: 18.6391 sec, Search Time: 0.5024 sec, Total Time: 19.1415 sec
Leaf Size: 500, Build Time: 17.4110 sec, Search Time: 0.4221 sec, Total Time: 17.8331 sec
Leaf Size: 600, Build Time: 17.5455 sec, Search Time: 0.4321 sec, Total Time: 17.9776 sec
Leaf Size: 700, Build Time: 17.1114 sec, Search Time: 0.4202 sec, Total Time: 17.5315 sec
Leaf Size: 800, Build Time: 17.9328 sec, Search Time: 0.4019 sec, Total Time: 18.3347 sec
Leaf Size: 900, Build Time: 17.9077 sec, Search Time: 0.4313 sec, Total Time: 18.33

**-----Opt Node Size-----**

In [13]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000])

# Total times from two runs (or versions)
total_times_1 = np.array([20.2138,18.5951,18.2371,17.4445,16.6252,17.5913,16.8138,16.9678,16.9135,
 15.8425,16.4243,15.9872,16.1823,15.8079,15.9574,16.2187,16.0911,16.135 ,
 16.1472,14.6683,15.0108,15.0249,14.5102,14.4799,15.9504,15.3897,15.4199,
 15.3986,15.5213,15.2959,15.1427,15.2728,15.646 ,16.0292,16.332 ,15.5517,
 15.6112,15.2975,15.5609,14.5761,14.9449,14.7575,14.4979,14.6495,14.6918,
 14.848 ,14.1731,14.3971,14.5328,14.6661,14.2703,13.7114,13.4274,13.2705,
 13.2637,13.7623,13.5286,13.2539,13.1116,13.1747,13.3047,13.9732,13.7787,
 13.4961,13.302 ,13.503 ,13.9585,13.7692,13.6602,13.6189,14.1607,14.0159,
 13.6375,13.6372,13.5146,13.5372,13.4079,13.9508,12.5472,13.225 ,12.7939,
 12.9522,12.8751,13.1714,13.2619,12.973 ,13.0748,13.4288,13.6154,13.3435,
 13.0072,12.9992,13.2775,13.3059,13.512 ,13.834 ,13.4336,13.2219,13.0817,
 12.9427])

total_times_2 = np.array([22.1287,19.5609,18.7596,19.1415,17.8331,17.9776,17.5315,18.3347,18.3391,
 16.5023,16.8453,16.821 ,16.4963,16.2553,16.4618,16.7017,16.8083,16.819 ,
 16.7869,15.8364,15.2431,15.6408,16.4078,15.8911,15.9707,16.099 ,15.8911,
 15.9622,15.5229,15.8446,16.4014,16.2929,16.0657,15.9423,15.7548,16.0449,
 16.0568,15.6893,15.5781,14.8296,15.1846,14.7258,15.0155,14.7334,14.7083,
 14.1966,13.5986,14.0555,14.1024,14.0559,14.5685,14.7789,15.8272,14.8973,
 14.2507,13.9594,14.4129,14.5707,14.784 ,14.3684,14.2211,14.5459,14.3146,
 13.9972,13.8691,13.8994,14.0095,14.1141,14.1296,14.2641,13.888 ,13.483 ,
 13.6117,13.5945,14.0212,14.0307,13.9686,14.0169,13.8411,13.6172,13.246 ,
 13.1859,13.1138,12.732 ,12.5051,12.9955,12.7951,13.0976,12.4722,12.5319,
 12.2895,12.8511,13.6344,13.0408,13.1832,13.4228,13.152 ,12.7711,13.0776,
 13.1904])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 12.6484
Minimum found at leaf size: 9100


**-----------Epsilon Optimization For Rebuild---------**

**---------------Run 1---------------------**

In [16]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9100
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 61.70it/s]


-------------------------------------------------------------
Build Time: 13.5144 sec, Search Time: 0.4853 sec
Epsilon: 0.0, Total Time: 13.9997 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 334.64it/s]


-------------------------------------------------------------
Build Time: 3.7387 sec, Search Time: 0.0745 sec
Epsilon: 0.5, Total Time: 3.8132 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.0439 sec, Search Time: 0.0027 sec
Epsilon: 24.0, Total Time: 2.0466 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 45764.36it/s]


-------------------------------------------------------------
Build Time: 1.9912 sec, Search Time: 0.0023 sec
Epsilon: 24.5, Total Time: 1.9934 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 26370.98it/s]

-------------------------------------------------------------
Build Time: 1.9490 sec, Search Time: 0.0031 sec
Epsilon: 25.0, Total Time: 1.9521 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.5144 sec, Search Time: 0.4853 sec, Total Time: 13.9997 sec
Epsilon: 0.5, Build Time: 3.7387 sec, Search Time: 0.0745 sec, Total Time: 3.8132 sec
Epsilon: 1.0, Build Time: 2.3825 sec, Search Time: 0.0214 sec, Total Time: 2.4039 sec
Epsilon: 1.5, Build Time: 2.0544 sec, Search Time: 0.0104 sec, Total Time: 2.0648 sec
Epsilon: 2.0, Build Time: 1.8777 sec, Search Time: 0.0123 sec, Total Time: 1.8900 sec
Epsilon: 2.5, Build Time: 2.0062 sec, Search Time: 0.0053 sec, Total Time: 2.0115 sec
Epsilon: 3.0, Build Time: 2.0912 sec, Search Time: 0.0044 sec, Total Time: 2.0956 sec
Epsilon: 3.5, Build Time: 2.0277 sec, Search Time: 0.0037 sec, Total Time: 2.0313 sec
Epsilon: 4.0, Build Time: 2.0543 sec, Search Time: 0.0039 sec, Total Time: 2.0581 sec
Epsilon: 4.5, Build Time: 2.01

**-------------Run 2------------------------**

In [18]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size = 9100
# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data = dataset[:data_size]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    vptree = VPTree(leaf_size=leaf_size, epsilon=epsilon)
    vptree.build(current_data)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 70.44it/s]


-------------------------------------------------------------
Build Time: 13.7798 sec, Search Time: 0.4430 sec
Epsilon: 0.0, Total Time: 14.2228 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 335.95it/s]


-------------------------------------------------------------
Build Time: 3.6396 sec, Search Time: 0.0746 sec
Epsilon: 0.5, Total Time: 3.7141 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 2.0013 sec, Search Time: 0.0038 sec
Epsilon: 24.0, Total Time: 2.0051 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 62930.29it/s]


-------------------------------------------------------------
Build Time: 1.9591 sec, Search Time: 0.0024 sec
Epsilon: 24.5, Total Time: 1.9615 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 57143.11it/s]

-------------------------------------------------------------
Build Time: 1.9110 sec, Search Time: 0.0030 sec
Epsilon: 25.0, Total Time: 1.9140 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 13.7798 sec, Search Time: 0.4430 sec, Total Time: 14.2228 sec
Epsilon: 0.5, Build Time: 3.6396 sec, Search Time: 0.0746 sec, Total Time: 3.7141 sec
Epsilon: 1.0, Build Time: 2.3481 sec, Search Time: 0.0182 sec, Total Time: 2.3663 sec
Epsilon: 1.5, Build Time: 2.0551 sec, Search Time: 0.0115 sec, Total Time: 2.0666 sec
Epsilon: 2.0, Build Time: 1.8946 sec, Search Time: 0.0111 sec, Total Time: 1.9057 sec
Epsilon: 2.5, Build Time: 1.9934 sec, Search Time: 0.0053 sec, Total Time: 1.9987 sec
Epsilon: 3.0, Build Time: 2.0262 sec, Search Time: 0.0044 sec, Total Time: 2.0306 sec
Epsilon: 3.5, Build Time: 1.9545 sec, Search Time: 0.0041 sec, Total Time: 1.9586 sec
Epsilon: 4.0, Build Time: 2.0415 sec, Search Time: 0.0036 sec, Total Time: 2.0452 sec
Epsilon: 4.5, Build Time: 1.99

**-----------Epsilon Optimization For Rebuild------------**

In [19]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ])

# Total times from two runs (or versions)
total_times_1 = np.array([14.2228, 3.7141, 2.3663, 2.0666, 1.9057, 1.9987, 2.0306, 1.9586, 2.0452,
  1.9957, 1.9727, 2.0086, 1.9885, 2.0519, 2.0441, 2.0037, 2.0032, 2.0509,
  1.9846, 2.0302, 2.0005, 1.9724, 1.9797, 1.9375, 2.0704, 2.1194, 2.0539,
  1.9696, 1.9936, 2.016 , 1.9627, 2.0411, 2.0196, 2.026 , 2.0159, 2.0006,
  2.0871, 2.0483, 1.9714, 1.9614, 1.979 , 2.0051, 2.0263, 1.996 , 2.0582,
  1.9998, 1.9893, 1.9665, 2.0051, 1.9615, 1.914 ])

total_times_2 = np.array([13.9997, 3.8132, 2.4039, 2.0648, 1.89  , 2.0115, 2.0956, 2.0313, 2.0581,
  2.0178, 2.054 , 2.0646, 1.9654, 1.9336, 1.9981, 1.9486, 1.9631, 2.0199,
  2.0821, 1.9759, 1.9593, 1.9744, 1.9454, 2.0608, 2.0276, 1.9714, 1.9652,
  1.956 , 2.0113, 2.0411, 2.0122, 1.9867, 1.9539, 1.9874, 1.9894, 2.0275,
  1.9869, 2.043 , 1.9914, 1.9755, 2.0503, 2.0257, 1.9346, 1.9507, 1.9879,
  2.0289, 2.048 , 1.9799, 2.0466, 1.9934, 1.9521])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 1.8979
Minimum found at epsilon size: 2.0


**-------------VP Node Optimization For Insert--------**

**--------------Run 1------------------**

In [20]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the VP Tree With Insertions---
    vptree.leaf_size = leaf_size
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 21.91it/s]


-------------------------------------------------------------
Build Time: 8.6920 sec, Search Time: 1.1024 sec
Leaf Size: 100, Total Time: 9.7944 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 22.84it/s]


-------------------------------------------------------------
Build Time: 8.6259 sec, Search Time: 1.0831 sec
Leaf Size: 200, Total Time: 9.7090 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.7385 sec, Search Time: 1.0951 sec
Leaf Size: 9,800, Total Time: 9.8335 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 21.31it/s]


-------------------------------------------------------------
Build Time: 8.4853 sec, Search Time: 1.1689 sec
Leaf Size: 9,900, Total Time: 9.6542 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 21.97it/s]

-------------------------------------------------------------
Build Time: 8.6508 sec, Search Time: 1.1406 sec
Leaf Size: 10,000, Total Time: 9.7913 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 8.6920 sec, Search Time: 1.1024 sec, Total Time: 9.7944 sec
Leaf Size: 200, Build Time: 8.6259 sec, Search Time: 1.0831 sec, Total Time: 9.7090 sec
Leaf Size: 300, Build Time: 8.6633 sec, Search Time: 1.0630 sec, Total Time: 9.7263 sec
Leaf Size: 400, Build Time: 8.5145 sec, Search Time: 1.0791 sec, Total Time: 9.5936 sec
Leaf Size: 500, Build Time: 8.5377 sec, Search Time: 1.0796 sec, Total Time: 9.6173 sec
Leaf Size: 600, Build Time: 8.6936 sec, Search Time: 1.0548 sec, Total Time: 9.7484 sec
Leaf Size: 700, Build Time: 8.6602 sec, Search Time: 1.0693 sec, Total Time: 9.7296 sec
Leaf Size: 800, Build Time: 8.6063 sec, Search Time: 1.0679 sec, Total Time: 9.6742 sec
Leaf Size: 900, Build Time: 8.4592 sec, Search Time: 1.0771 sec, Total Time: 9.5363 sec
Leaf Size: 1,

**-------------Run 2---------------**

In [22]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

# Define leaf size range
leaf_size_range_start = 100
leaf_size_range_end = 10_000  # Ensure inclusive range
leaf_size_step = 100

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

data_size, query_size = test_combinations[0]

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, leaf_size in enumerate(range(leaf_size_range_start, leaf_size_range_end + leaf_size_step, leaf_size_step)):
    print(f"\n{idx+1}: === Testing with Leaf Size: {leaf_size:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]
    # --- Build the VP Tree With Insertions---
    vptree.leaf_size = leaf_size
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((leaf_size, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Leaf Size: {leaf_size:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Node Size Test ===")
for l_size, b_time, s_time, t_time in results:
    print(f"Leaf Size: {l_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Leaf Size: 100, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 22.86it/s]


-------------------------------------------------------------
Build Time: 8.5726 sec, Search Time: 1.0968 sec
Leaf Size: 100, Total Time: 9.6694 sec

2: === Testing with Leaf Size: 200, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 22.74it/s]


-------------------------------------------------------------
Build Time: 8.6166 sec, Search Time: 1.0970 sec
Leaf Size: 200, Total Time: 9.7136 sec

3: === Testing with Leaf Size: 300, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.5697 sec, Search Time: 1.1115 sec
Leaf Size: 9,800, Total Time: 9.6812 sec

99: === Testing with Leaf Size: 9,900, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 22.70it/s]


-------------------------------------------------------------
Build Time: 8.6851 sec, Search Time: 1.1190 sec
Leaf Size: 9,900, Total Time: 9.8041 sec

100: === Testing with Leaf Size: 10,000, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 22.96it/s]

-------------------------------------------------------------
Build Time: 8.7212 sec, Search Time: 1.1145 sec
Leaf Size: 10,000, Total Time: 9.8357 sec

=== Summary of VP Tree Node Size Test ===
Leaf Size: 100, Build Time: 8.5726 sec, Search Time: 1.0968 sec, Total Time: 9.6694 sec
Leaf Size: 200, Build Time: 8.6166 sec, Search Time: 1.0970 sec, Total Time: 9.7136 sec
Leaf Size: 300, Build Time: 8.6670 sec, Search Time: 1.0984 sec, Total Time: 9.7655 sec
Leaf Size: 400, Build Time: 8.6879 sec, Search Time: 1.1186 sec, Total Time: 9.8065 sec
Leaf Size: 500, Build Time: 8.7309 sec, Search Time: 1.1108 sec, Total Time: 9.8418 sec
Leaf Size: 600, Build Time: 8.7525 sec, Search Time: 1.1274 sec, Total Time: 9.8799 sec
Leaf Size: 700, Build Time: 8.5105 sec, Search Time: 1.1242 sec, Total Time: 9.6347 sec
Leaf Size: 800, Build Time: 8.6389 sec, Search Time: 1.1205 sec, Total Time: 9.7595 sec
Leaf Size: 900, Build Time: 8.5313 sec, Search Time: 1.0912 sec, Total Time: 9.6225 sec
Leaf Size: 1,

**------Optimum Node Size for Insertion----**

In [23]:
import numpy as np

# Leaf sizes
leaf_sizes = np.array([  100,  200,  300,  400,  500,  600,  700,  800,  900, 1000, 1100, 1200,
  1300, 1400, 1500, 1600, 1700, 1800, 1900, 2000, 2100, 2200, 2300, 2400,
  2500, 2600, 2700, 2800, 2900, 3000, 3100, 3200, 3300, 3400, 3500, 3600,
  3700, 3800, 3900, 4000, 4100, 4200, 4300, 4400, 4500, 4600, 4700, 4800,
  4900, 5000, 5100, 5200, 5300, 5400, 5500, 5600, 5700, 5800, 5900, 6000,
  6100, 6200, 6300, 6400, 6500, 6600, 6700, 6800, 6900, 7000, 7100, 7200,
  7300, 7400, 7500, 7600, 7700, 7800, 7900, 8000, 8100, 8200, 8300, 8400,
  8500, 8600, 8700, 8800, 8900, 9000, 9100, 9200, 9300, 9400, 9500, 9600,
  9700, 9800, 9900,10000])

# Total times from two runs (or versions)
total_times_1 = np.array([9.7944,9.709 ,9.7263,9.5936,9.6173,9.7484,9.7296,9.6742,9.5363,9.6399,
 9.6987,9.6174,9.6217,9.6005,9.6161,9.5303,9.6708,9.6622,9.6626,9.7426,
 9.7399,9.7235,9.8355,9.6745,9.7513,9.6862,9.7665,9.6352,9.7693,9.7526,
 9.7471,9.6865,9.7751,9.647 ,9.6211,9.6988,9.7218,9.6653,9.6741,9.639 ,
 9.8599,9.724 ,9.6759,9.6118,9.6054,9.6308,9.9143,9.7215,9.7108,9.7506,
 9.6369,9.7963,9.7555,9.74  ,9.8593,9.8065,9.7526,9.7926,9.7802,9.7018,
 9.7235,9.695 ,9.7897,9.8429,9.8316,9.6908,9.5826,9.6477,9.7173,9.9233,
 9.7802,9.7113,9.8179,9.8895,9.6792,9.7321,9.7817,9.6446,9.6907,9.7752,
 9.6844,9.7332,9.6808,9.7975,9.5727,9.6614,9.6684,9.6326,9.6767,9.7063,
 9.7203,9.7732,9.7591,9.8903,9.6794,9.7444,9.6971,9.8335,9.6542,9.7913])

total_times_2 = np.array([ 9.6694, 9.7136, 9.7655, 9.8065, 9.8418, 9.8799, 9.6347, 9.7595, 9.6225,
  9.5728, 9.6001, 9.6587, 9.8062, 9.8373, 9.717 , 9.818 , 9.7015, 9.6971,
  9.707 , 9.845 , 9.645 ,10.103 ,10.0788, 9.7185, 9.6298, 9.7778, 9.786 ,
  9.7773, 9.7762, 9.6827, 9.6718, 9.6447, 9.6757, 9.7778, 9.678 , 9.8204,
  9.7785, 9.7802, 9.6325, 9.784 , 9.5885, 9.6685, 9.6963, 9.7576, 9.7001,
  9.8879, 9.7078, 9.9658, 9.7276, 9.7215, 9.7431, 9.8245, 9.672 , 9.8038,
  9.7913, 9.6541, 9.6851, 9.8628, 9.6842, 9.8533, 9.7445, 9.6581, 9.618 ,
  9.6675, 9.6652, 9.7598, 9.7315, 9.7828, 9.7074, 9.7599, 9.6083, 9.7335,
  9.5848, 9.6127, 9.7894, 9.8289, 9.6321, 9.7201, 9.6865, 9.7309, 9.7448,
  9.952 , 9.824 , 9.7957, 9.7187, 9.705 , 9.782 , 9.6795, 9.7753, 9.9799,
  9.9879, 9.84  , 9.8066, 9.7709, 9.7712, 9.7336, 9.6129, 9.6812, 9.8041,
  9.8357])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_leaf_size = leaf_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at leaf size: {optimal_leaf_size}")


Average total time (sec): 9.5794
Minimum found at leaf size: 900


**------Epsilon Optimization For Insertion------------**

**----------Run 1-----------------**

In [24]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

leaf_size_insert = 900

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.leaf_size = leaf_size_insert
    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 24.45it/s]


-------------------------------------------------------------
Build Time: 8.5365 sec, Search Time: 1.1284 sec
Epsilon: 0.0, Total Time: 9.6649 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 28.43it/s]


-------------------------------------------------------------
Build Time: 8.6169 sec, Search Time: 1.0318 sec
Epsilon: 0.5, Total Time: 9.6488 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.6756 sec, Search Time: 1.1507 sec
Epsilon: 24.0, Total Time: 9.8262 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 22.70it/s]


-------------------------------------------------------------
Build Time: 8.6446 sec, Search Time: 1.1340 sec
Epsilon: 24.5, Total Time: 9.7786 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 22.67it/s]

-------------------------------------------------------------
Build Time: 8.8174 sec, Search Time: 1.1305 sec
Epsilon: 25.0, Total Time: 9.9479 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.5365 sec, Search Time: 1.1284 sec, Total Time: 9.6649 sec
Epsilon: 0.5, Build Time: 8.6169 sec, Search Time: 1.0318 sec, Total Time: 9.6488 sec
Epsilon: 1.0, Build Time: 8.6040 sec, Search Time: 0.9822 sec, Total Time: 9.5862 sec
Epsilon: 1.5, Build Time: 8.6063 sec, Search Time: 1.0179 sec, Total Time: 9.6242 sec
Epsilon: 2.0, Build Time: 8.5343 sec, Search Time: 1.1959 sec, Total Time: 9.7302 sec
Epsilon: 2.5, Build Time: 8.6359 sec, Search Time: 1.1767 sec, Total Time: 9.8125 sec
Epsilon: 3.0, Build Time: 8.5895 sec, Search Time: 1.1674 sec, Total Time: 9.7569 sec
Epsilon: 3.5, Build Time: 8.6580 sec, Search Time: 1.1767 sec, Total Time: 9.8347 sec
Epsilon: 4.0, Build Time: 8.7062 sec, Search Time: 1.1585 sec, Total Time: 9.8647 sec
Epsilon: 4.5, Build Time: 8.6728

**----------Run 2---------------**

In [26]:
import time
from tqdm import tqdm

# Test code for VP Tree
results = []

leaf_size_rebuild = 9100
epsilon_rebuild = 2.0

leaf_size_insert = 900

# Define epsilon range
epsilon_range_start = 0
epsilon_range_end = 25  # Ensure inclusive range
epsilon_size_step = 0.5

# Obtain Dataset Sample
current_data_rebuild = dataset[:MB]  # Directly slicing for clarity
# Readjust for the insertion
data_size, query_size = test_combinations[0]

# Retrieve the pre-generated queries from the list using the same order
queries = queries_list[0]  # Use first query set, as per your specification

for idx, epsilon in enumerate(np.arange(epsilon_range_start, epsilon_range_end + epsilon_size_step, epsilon_size_step)):
    epsilon = round(epsilon, 10)
    print(f"\n{idx+1}: === Testing with Epsilon: {epsilon:,}, Data Size: {data_size:,} points, Query Size: {query_size:,} queries ===")

    vptree = VPTree(leaf_size=leaf_size_rebuild, epsilon=epsilon_rebuild)
    vptree.build(current_data_rebuild)

    # Obtain Dataset Sample
    current_data = dataset[MB:data_size+MB]

    vptree.leaf_size = leaf_size_insert
    vptree.epsilon = epsilon

    # --- Build the VP Tree ---
    build_start = time.perf_counter()
    for pt in tqdm(current_data, desc="Inserting VP Tree"):
        vptree.insert(pt)
    build_time = time.perf_counter() - build_start

    # --- Perform the Queries ---
    start_time = time.perf_counter()
    vptree.prepare_for_search()
    for query in tqdm(queries, desc=f"Querying (Epsilon {epsilon:,})"):
        r_min, nn_pt = vptree.search(query=query)
    end_time = time.perf_counter()
    search_time = end_time - start_time

    total_time = build_time + search_time
    results.append((epsilon, build_time, search_time, total_time))

    print("-------------------------------------------------------------")
    print(f"Build Time: {build_time:.4f} sec, Search Time: {search_time:.4f} sec")
    print(f"Epsilon: {epsilon:,}, Total Time: {total_time:.4f} sec")

print("\n=== Summary of VP Tree Epsilon Size Test ===")
for e_size, b_time, s_time, t_time in results:
    print(f"Epsilon: {e_size:,}, Build Time: {b_time:.4f} sec, Search Time: {s_time:.4f} sec, Total Time: {t_time:.4f} sec")



1: === Testing with Epsilon: 0.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.0): 100%|██████████| 20/20 [00:00<00:00, 22.69it/s]


-------------------------------------------------------------
Build Time: 8.6358 sec, Search Time: 1.1897 sec
Epsilon: 0.0, Total Time: 9.8255 sec

2: === Testing with Epsilon: 0.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 0.5): 100%|██████████| 20/20 [00:00<00:00, 23.52it/s]


-------------------------------------------------------------
Build Time: 8.5062 sec, Search Time: 1.1842 sec
Epsilon: 0.5, Total Time: 9.6905 sec

3: === Testing with Epsilon: 1.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


...

-------------------------------------------------------------
Build Time: 8.6530 sec, Search Time: 1.1513 sec
Epsilon: 24.0, Total Time: 9.8043 sec

50: === Testing with Epsilon: 24.5, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 24.5): 100%|██████████| 20/20 [00:00<00:00, 21.92it/s]


-------------------------------------------------------------
Build Time: 8.4800 sec, Search Time: 1.1526 sec
Epsilon: 24.5, Total Time: 9.6325 sec

51: === Testing with Epsilon: 25.0, Data Size: 1,000,000 points, Query Size: 20 queries ===


Querying (Epsilon 25.0): 100%|██████████| 20/20 [00:00<00:00, 21.96it/s]

-------------------------------------------------------------
Build Time: 8.7096 sec, Search Time: 1.1648 sec
Epsilon: 25.0, Total Time: 9.8744 sec

=== Summary of VP Tree Epsilon Size Test ===
Epsilon: 0.0, Build Time: 8.6358 sec, Search Time: 1.1897 sec, Total Time: 9.8255 sec
Epsilon: 0.5, Build Time: 8.5062 sec, Search Time: 1.1842 sec, Total Time: 9.6905 sec
Epsilon: 1.0, Build Time: 8.5793 sec, Search Time: 1.1168 sec, Total Time: 9.6961 sec
Epsilon: 1.5, Build Time: 8.9096 sec, Search Time: 1.1736 sec, Total Time: 10.0832 sec
Epsilon: 2.0, Build Time: 8.5945 sec, Search Time: 1.1463 sec, Total Time: 9.7409 sec
Epsilon: 2.5, Build Time: 8.7947 sec, Search Time: 1.1322 sec, Total Time: 9.9269 sec
Epsilon: 3.0, Build Time: 8.5747 sec, Search Time: 1.2790 sec, Total Time: 9.8537 sec
Epsilon: 3.5, Build Time: 8.4894 sec, Search Time: 1.1731 sec, Total Time: 9.6624 sec
Epsilon: 4.0, Build Time: 8.4547 sec, Search Time: 1.1704 sec, Total Time: 9.6251 sec
Epsilon: 4.5, Build Time: 8.659

**--------Epsilon Optimization For Insert---------**

In [27]:
import numpy as np

# Epsilon sizes
epsilon_sizes = np.array([ 0. , 0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. , 5.5, 6. , 6.5,
  7. , 7.5, 8. , 8.5, 9. , 9.5,10. ,10.5,11. ,11.5,12. ,12.5,13. ,13.5,
 14. ,14.5,15. ,15.5,16. ,16.5,17. ,17.5,18. ,18.5,19. ,19.5,20. ,20.5,
 21. ,21.5,22. ,22.5,23. ,23.5,24. ,24.5,25. ])

# Total times from two runs (or versions)
total_times_1 = np.array([ 9.6649, 9.6488, 9.5862, 9.6242, 9.7302, 9.8125, 9.7569, 9.8347, 9.8647,
  9.8307, 9.9524,10.0283, 9.8226, 9.7905, 9.7216, 9.7257, 9.8637, 9.7498,
  9.6662, 9.6773, 9.677 , 9.6104, 9.547 , 9.4913, 9.5597, 9.6643, 9.6308,
  9.6267, 9.741 , 9.6249, 9.6151, 9.553 , 9.6821, 9.6449, 9.721 , 9.7162,
  9.644 , 9.6424, 9.6623, 9.7087, 9.7098, 9.8121, 9.7226, 9.6677, 9.7277,
  9.6854, 9.8458, 9.8368, 9.8262, 9.7786, 9.9479])

total_times_2 = np.array([ 9.8255, 9.6905, 9.6961,10.0832, 9.7409, 9.9269, 9.8537, 9.6624, 9.6251,
  9.8288, 9.8539, 9.7478, 9.8848, 9.8564, 9.9225, 9.7928, 9.7121, 9.7581,
  9.7076, 9.7465, 9.7499, 9.9643, 9.7513, 9.7791, 9.6319, 9.7853, 9.7969,
 10.056 , 9.6962, 9.8318, 9.7096, 9.6892, 9.7103, 9.8291, 9.8619, 9.6784,
  9.7803, 9.7454, 9.8607, 9.8285, 9.7944, 9.7506, 9.8026, 9.7758, 9.8722,
  9.9219, 9.7792, 9.8029, 9.8043, 9.6325, 9.8744])

# Average total time across the two runs
avg_total_times = (total_times_1 + total_times_2) / 2

# Index of the minimum average total time
min_idx = np.argmin(avg_total_times)

# Corresponding leaf size
optimal_epsilon_size = epsilon_sizes[min_idx]

# Print results
print(f"Average total time (sec): {avg_total_times[min_idx]:.4f}")
print(f"Minimum found at epsilon size: {optimal_epsilon_size}")


Average total time (sec): 9.5958
Minimum found at epsilon size: 12.0
